In [94]:
%run First_Summarizer.ipynb

## Importation d'un modèle pour la langue francaise

In [11]:
import spacy
path = "C:\\Users\\Ivaros\\Documents\\Python\\NLP\\" #A modifier/automatiser
nlp = spacy.load(path + "fr_core_news_sm-2.2.5") #Charge le modele fr deja installe

## Lecture du texte

In [12]:
with open('C:\\Users\\Ivaros\\Documents\\Python\\Stage_3A\\Debat\\Interview_Prelats.txt', 'r', encoding = "utf-8") as file:
     prelat = file.read() 

# Améliorations du Premier Summarizer

### Sur la Tokenisation

In [23]:
## Seconde methode pour Tokeniser. fonction sent_tokenize
import nltk
from nltk.tokenize import sent_tokenize
#nltk.download()
Tokenize = sent_tokenize(prelat, language = "french")
Tokenize
len(Tokenize)

71

La tokenisation semble de meilleurs qualités !

## Suppression des StopWords

In [ ]:
L_supp =["Bonjour", "bonjour"] #Liste d'element a supprimer

In [17]:
#Pour supprimer les stopWords
#nltk.download('stopwords')
from nltk.corpus import stopwords

stopWords = set(stopwords.words('french'))

def delete_SW(test,stopWords) :
    clean_words = []
    for token in return_token(test): #return_token tokenize mot a mot
        if token not in stopWords:
            clean_words.append(token)
    return clean_words

#clean_words = delete_SW(prelat,stopWords)
#clean_words #Renvoie une tokenisation mot a mot

#Ajout d'elements a SW

def add_SW(L,SW) :
    """Ajoute une liste d element"""
    for j in L:
       SW.add(j) 
    return SW

stopWords

{'ai',
 'aie',
 'aient',
 'aies',
 'ait',
 'as',
 'au',
 'aura',
 'aurai',
 'auraient',
 'aurais',
 'aurait',
 'auras',
 'aurez',
 'auriez',
 'aurions',
 'aurons',
 'auront',
 'aux',
 'avaient',
 'avais',
 'avait',
 'avec',
 'avez',
 'aviez',
 'avions',
 'avons',
 'ayant',
 'ayante',
 'ayantes',
 'ayants',
 'ayez',
 'ayons',
 'c',
 'ce',
 'ces',
 'd',
 'dans',
 'de',
 'des',
 'du',
 'elle',
 'en',
 'es',
 'est',
 'et',
 'eu',
 'eue',
 'eues',
 'eurent',
 'eus',
 'eusse',
 'eussent',
 'eusses',
 'eussiez',
 'eussions',
 'eut',
 'eux',
 'eûmes',
 'eût',
 'eûtes',
 'furent',
 'fus',
 'fusse',
 'fussent',
 'fusses',
 'fussiez',
 'fussions',
 'fut',
 'fûmes',
 'fût',
 'fûtes',
 'il',
 'ils',
 'j',
 'je',
 'l',
 'la',
 'le',
 'les',
 'leur',
 'lui',
 'm',
 'ma',
 'mais',
 'me',
 'mes',
 'moi',
 'mon',
 'même',
 'n',
 'ne',
 'nos',
 'notre',
 'nous',
 'on',
 'ont',
 'ou',
 'par',
 'pas',
 'pour',
 'qu',
 'que',
 'qui',
 's',
 'sa',
 'se',
 'sera',
 'serai',
 'seraient',
 'serais',
 'serait',


In [83]:
#Relancer la fonction de resume avec les cgts

def return_text_emb(texte) : 
    """Vectorisation du texte"""
    text_vect = []
    Tokenize = sent_tokenize(texte, language = "french") #Tokenisation par phrase    
    for i in range(len(Tokenize)) :
        Sentence_emb = return_word_embedding(Tokenize[i]) #Vectorisation des phrases
        vect = return_sentence_emb(Sentence_emb)
        text_vect += [vect]
    return text_vect

Test = return_text_emb(prelat)

## Pondération de la phrase

On va chercher a calculer la frequence d'apparition de chaque mot dans notre texte pour donner de l importance aux mots "rares"

### Obtention de la fréquence


In [7]:
#Récupération des racines de mots

from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer(language='french')

def return_stem(sentence):
    doc = nlp(sentence)
    return [stemmer.stem(X.text) for X in doc] #X.text correspond au mot



In [8]:
def obt_occ(dictionnaire,stopwords,mot) :
    """Renvoie le nombre d'occurence du mot """
    if mot in stopwords :
        return 0
    else : 
        if (mot in dictionnaire) :
            return 1/dictionnaire[mot]
        return 1

In [18]:
#Fonctionne, mais doit tourner sur plusieurs textes pour etre efficace ?
def word_in_text(texte):
    """Renvoie un dictionnaire avec en cle
    les mots du texte et en valeur le nombre d'occurence"""
    dictionnaire = {}
    tokens = return_stem(texte) #Recuperation des racines des tokens
    n = len(tokens)
    for i in range(len(tokens)): #Pour tout les tokens 
        if (tokens[i] in dictionnaire):
            dictionnaire[tokens[i]] = dictionnaire[tokens[i]] + (1/n)
        else:
            dictionnaire[tokens[i]] = (1/n)
    return dictionnaire

occ_tot = word_in_text(prelat)

0.009453781512605041

In [ ]:
#Calculer l'embedding de la phrase a partir des poids. poids x representation a partir de la racine

In [51]:
#sentence correspond a la str de la phrase
def calc_emb_sent(sentence,occ_tot):
    """Prend en entree une phrase vectorise mot a mot et renvoie une avec ponderation"""
    sentence_emb = return_word_embedding(sentence)
    somme = sentence_emb[0] - sentence_emb[0] #Initialisation d'un tableau vide
    tokens = return_stem(sentence) #Recuperation des racines des tokens
    n = len(tokens)
    for i in range(len(tokens)): #Pour tout les tokens
        poids = occ_tot[str(tokens[i])]
        somme += poids * sentence_emb[i]
    return somme/n

In [87]:
def calc_emb_text(texte,occ_tot):
    """texte en entree et renvoie l embedding du texte avec ponderation"""
    text = []
    Tokenize = sent_tokenize(texte, language = "french")
    for i in range(len(Tokenize)): 
        text += [calc_emb_sent(Tokenize[i],occ_tot)]
    return text
        
A = calc_emb_text(prelat,occ_tot)

71

In [88]:
def clusterisation2(texte, occ_tot, percent = 0.1) :
    """K_means sur le texte, percent correspond au pourcentage du texte que l'on veut garder"""
    #Obtention du nombre de phrase
    Tokenize = sent_tokenize(texte, language = "french")
    nb_phrase = len(Tokenize)
    nb_centre = int(np.ceil(percent*nb_phrase))
    #Representation vectorielle du texte phrase par phrase
    emb_text = calc_emb_text(texte,occ_tot)
    #Kmeans
    kmeans = KMeans(n_clusters=nb_centre)
    kmeans = kmeans.fit(emb_text)
    return (kmeans, nb_centre)

In [91]:
def print_summary2(texte, occ_tot) :
    """Renvoie le resume"""
    clust = clusterisation2(texte, occ_tot)
    kmeans  = clust[0]
    nb_centre = clust[1]
    avg = []
    for j in range(nb_centre):
        idx = np.where(kmeans.labels_ == j)[0]
        avg.append(np.mean(idx))
    Tokenize = sent_tokenize(texte, language = "french")
    emb_text = calc_emb_text(texte,occ_tot)
    closest, _ = pairwise_distances_argmin_min(kmeans.cluster_centers_, emb_text)
    ordering = sorted(range(nb_centre), key=lambda k: avg[k])
    summary = ' '.join([Tokenize[closest[idx]] for idx in ordering])
    return summary

In [92]:
summary = print_summary2(prelat,occ_tot)
summary

'L’INFLUENCE DE L’OEUVRE DE J. K. ROWLING\nL’autrice le dit elle-même sur son site, «par certains aspects, cet univers pourrait plaire à ceux qui auraient aimé Harry Potter, Le Seigneur des anneaux, Twilight, Eragon… et beaucoup d’autres.» Car en plus de nous plonger dans un univers fantastique qui cache de multiples secrets, les livres de Charlotte Abécassis Weigel sont empreints de magie, de suspens, de trahisons… Et quoi de mieux pour avoir des informations sur un roman que d’avoir les explications de son autrice ? J’ai alors découvert les fan-fictions, des histoires écrites par des fans que l’on peut trouver sur Internet. Il y en a beaucoup sur Harry Potter et j’ai tout de suite été séduite par l’accueil qu’elles recevaient. J’ai commencé en écrivant sur Harry Potter, puis, sans même m’en rendre compte, j’ai commencé à inventer mon univers. J’ai d’abord été attirée par la couverture très sobre du premier tome, un petit renard stylisé sur fond bleu, puis une fois le livre en main, j

In [96]:
print_summary(prelat)

'Aujourd’hui, je viens vous parler d’une saga d’heroic fantasy découverte lors de l’édition 2019 du Salon du Livre de Paris, Les Prélats de Faneas de Charlotte Abécassis Weigel.\n Je pouvais rire en lisant les descriptions du cou de Pétunia ou les maladresses d’Hagrid, et je pouvais frissonner en découvrant les détraqueurs, l’ En effet, l’autrice prend un malin plaisir à mener ses personnages, ainsi que ses lecteurs en bateau. Les raisons sont diverses.\n mélancolique des parents de Harry ou encore le sort réservé à Buck.\n De plus, c’est un temps fort dans la vie du livre, un moment d’échange et de partage, entre autre grâce aux cadeaux bonus débloqués à chaque fois que l’on dépasse un nouveau palier. Mes livres sortent effectivement via des financements participatifs sur la plateforme Ulule et non grâce à une maison d’édition. Crainte, réjouissance ?'

L'amélioration du process est concluante. Le résumé semble de meilleur qualité (Premier résumé). Cependant la mise en forme est à revoir !